2. Klasyfikacja grzybów (https://archive.ics.uci.edu/ml/datasets/Mushroom)
Przy użyciu danych mushrooms.csv zbuduj następujące modele: AdaBoost, XGBoost, LightGBM oraz CatBoost. Porównaj ich wyniki. Spróbuj ulepszyć wyniki każdego z wariantów poprzed dostrojenie parametrów - skorzystaj z dokumentacji wymienionych metod. 

In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [210]:
df = pd.read_csv("mushrooms.csv")

In [211]:
df.shape

(8124, 23)

In [212]:
df.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [214]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [215]:
df.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [216]:
features = df.drop("class", axis = 1)
features_dummies = pd.get_dummies(df.drop("class", axis = 1))
target = df["class"]

In [283]:
target = target.replace(('e', 'p'), (1, 0))

In [284]:
target

0       0
1       1
2       1
3       0
4       1
       ..
8119    1
8120    1
8121    1
8122    0
8123    1
Name: class, Length: 8124, dtype: int64

In [285]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_dummies, target, test_size = 0.9, random_state = 1, stratify = target)

In [286]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [287]:
def evaluate(prediction, y_test): 
    result = classification_report(y_test, prediction, output_dict = True)
    f1 = result['1']['f1-score']
    accuracy = result['accuracy']
    performance_data= {'f1-score':round(f1, 4),
                      'accuracy':round(accuracy, 4)}
    return performance_data

## Logistic Regression

In [288]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_pr = evaluate(lr_pred, y_test)
lr_pr

{'f1-score': 0.9991, 'accuracy': 0.999}

In [338]:
# l2 regularization + CV
from sklearn.model_selection import GridSearchCV
lr2 = LogisticRegression()
lr2_cv = GridSearchCV(lr2, {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]})
lr2_cv.fit(X_train, y_train)
print(lr2_cv.best_params_)
lr2_cv_pred = lr2_cv.predict(X_test)
lr2_cv_pr = evaluate(lr2_cv_pred, y_test)
lr2_cv_pr

{'C': 10}


{'f1-score': 0.9999, 'accuracy': 0.9999}

In [339]:
# l1 regularization + CV
lr3 = LogisticRegression(solver = 'liblinear', penalty = 'l1')
lr3_cv = GridSearchCV(lr3, {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]})
lr3_cv.fit(X_train, y_train)
print(lr3_cv.best_params_)
lr3_cv_pred = lr3_cv.predict(X_test)
lr3_cv_pr = evaluate(lr3_cv_pred, y_test)
lr3_cv_pr

{'C': 10}


{'f1-score': 0.9992, 'accuracy': 0.9992}

## Decision Tree

In [249]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
dtree_pred = dtree.predict(X_test)
dtree_pr = evaluate(dtree_pred, y_test)
dtree_pr

{'f1-score': 0.9992, 'accuracy': 0.9992}

## Random Forest

In [255]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_pr = evaluate(rf_pred, y_test)
rf_pr

{'f1-score': 1.0, 'accuracy': 1.0}

## SVM

In [313]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
svc_pr = evaluate(svc_pred, y_test)
svc_pr

{'f1-score': 0.9991, 'accuracy': 0.999}

In [343]:
# 'rbf' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc2 = SVC()
svc2_cv = GridSearchCV(svc2, param_grid)
svc2_cv.fit(X_train, y_train)
print(svc2_cv.best_params_)
svc2_cv_pred = svc2_cv.predict(X_test)
svc2_cv_pr = evaluate(svc2_cv_pred, y_test)
svc2_cv_pr

{'C': 1, 'gamma': 0.1}


{'f1-score': 0.9993, 'accuracy': 0.9993}

In [346]:
# 'linear' + CV
svc3 = SVC(kernel = 'linear')
svc3_cv = GridSearchCV(svc3, {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]})
svc3_cv.fit(X_train, y_train)
print(svc3_cv.best_params_)
svc3_cv_pred = svc3_cv.predict(X_test)
svc3_cv_pr = evaluate(svc3_cv_pred, y_test)
svc3_cv_pr

{'C': 1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [350]:
# 'poly' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc4 = SVC(kernel = 'poly')
svc4_cv = GridSearchCV(svc4, param_grid)
svc4_cv.fit(X_train, y_train)
print(svc4_cv.best_params_)
svc4_cv_pred = svc4_cv.predict(X_test)
svc4_cv_pr = evaluate(svc4_cv_pred, y_test)
svc4_cv_pr

{'C': 0.1, 'gamma': 0.1}


{'f1-score': 0.9991, 'accuracy': 0.999}

In [353]:
# 'sigmoid' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc5 = SVC(kernel = 'sigmoid')
svc5_cv = GridSearchCV(svc5, param_grid)
svc5_cv.fit(X_train, y_train)
print(svc5_cv.best_params_)
svc5_cv_pred = svc5_cv.predict(X_test)
svc5_cv_pr = evaluate(svc5_cv_pred, y_test)
svc5_cv_pr

{'C': 100, 'gamma': 0.01}


{'f1-score': 1.0, 'accuracy': 1.0}

## KNN

In [257]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_pr = evaluate(knn_pred, y_test)
knn_pr

{'f1-score': 0.9984, 'accuracy': 0.9984}

In [277]:
accuracies = {}
for neighbor in np.arange(1, 10):
    knn = KNeighborsClassifier(n_neighbors = neighbor)
    knn.fit(X_train, y_train)
    accuracies[neighbor] = knn.score(X_test, y_test)
accuracies

{1: 0.999863238512035,
 2: 0.999316192560175,
 3: 0.9983588621444202,
 4: 0.9980853391684902,
 5: 0.9982221006564551,
 6: 0.9967177242888403,
 7: 0.9975382932166302,
 8: 0.9943927789934355,
 9: 0.9953501094091903}

In [354]:
prediction_data = {'Model':['Logistic Regression', 'Logistic Regression - l2 regularization + CV', 'Logistic Regression - l1 regularization + CV', 'Decision Tree', 'Random Forest', 'SVM rbf', 'SVM rbf + CV', 'SVM linear + CV', 'SVM poly + CV', 'SVM sigmoid + CV' ,'KNN'],
                   'F1-score':[lr_pr['f1-score'], lr2_cv_pr['f1-score'], lr3_cv_pr['f1-score'], dtree_pr['f1-score'],rf_pr['f1-score'],svc_pr['f1-score'], svc2_cv_pr['f1-score'], svc3_cv_pr['f1-score'], svc4_cv_pr['f1-score'], svc5_cv_pr['f1-score'], knn_pr['f1-score']],
                   'Accuracy':[lr_pr['accuracy'], lr2_pr['accuracy'], lr3_pr['accuracy'], dtree_pr['accuracy'], rf_pr['accuracy'], svc_pr['accuracy'], svc2_cv_pr['accuracy'], svc3_cv_pr['accuracy'], svc4_cv_pr['accuracy'], svc5_cv_pr['accuracy'], knn_pr['accuracy']]
                    }
 
prediction_table = pd.DataFrame(prediction_data)
prediction_table

,Model,F1-score,Accuracy
0,Logistic Regression,0.9991,0.9990
1,Logistic Regression - l2 regularization + CV,0.9999,0.9999
2,Logistic Regression - l1 regularization + CV,0.9992,0.9999
3,Decision Tree,0.9992,0.9992
4,Random Forest,1.0000,1.0000
5,SVM rbf,0.9991,0.9990
6,SVM rbf + CV,0.9993,0.9993
7,SVM linear + CV,1.0000,1.0000
8,SVM poly + CV,0.9991,0.9990
9,SVM sigmoid + CV,1.0000,1.0000


## AdaBoost

In [266]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(random_state = 1)
ada.fit(X_train, y_train)
ada.score(X_test,y_test)

1.0

## XGBoost

In [269]:
!pip install xgboost

  Using cached xgboost-1.6.2-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl (1.7 MB)


In [270]:
import xgboost as xgb
xgboost = xgb.XGBClassifier(random_state = 1,learning_rate = 0.01)
xgboost.fit(X_train, y_train)
xgboost.score(X_test, y_test)

0.9971280087527352

## LightGBM

In [274]:
!pip install lightgbm

In [275]:
import lightgbm as lgb

OSError: dlopen(/Users/edytastodolkiewicz/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/edytastodolkiewicz/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)